In [3]:
%pwd
%cd ../..

/home/artur/PycharmProjects/conv-net-sandbox


In [4]:
import argparse
import sys
from datetime import datetime
from pathlib import Path

import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from dotenv import load_dotenv, find_dotenv
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TerminateOnNaN
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

from models import build_simple_conv_net

In [5]:
BASE_DIR = Path().resolve()
LOGS_DIR = BASE_DIR.joinpath('logs', f'{datetime.now().strftime("%m-%d-%Y_%H:%M:%S")}')

WEIGHTS_OUTPUT_PATH = LOGS_DIR.joinpath('ep{epoch:03d}-loss{loss:.4f}-val_loss{val_loss:.4f}.h5')
MODEL_OUTPUT_PATH = LOGS_DIR.joinpath('trained_model.h5')
DEFAULT_MODEL_INPUT_SIZE = '300x300'

ENV_FILE = find_dotenv()
if ENV_FILE:
    load_dotenv(ENV_FILE)

In [7]:
input_shape = np.asarray([400, 400, 3])
checkpoint = ModelCheckpoint(str(WEIGHTS_OUTPUT_PATH),
                             monitor='val_loss',
                             verbose=1,
                             save_weights_only=True,
                             save_best_only=True)
logging = TensorBoard(log_dir=str(LOGS_DIR))
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=300, verbose=1)
terminate_on_nan = TerminateOnNaN()
callbacks = [logging, checkpoint, early_stopping, terminate_on_nan]

optimizer = Adam()
model = build_simple_conv_net(input_shape)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
epochs = 30

In [9]:
train_datagen = ImageDataGenerator(rescale=1 / 255)
test_datagen = ImageDataGenerator(rescale=1 / 255)

train_generator = train_datagen.flow_from_directory(
    '/var/data/Downloads/archive/chest_xray/chest_xray/train/',
    target_size=input_shape[:2],
    batch_size=16,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    '/var/data/Downloads/archive/chest_xray/chest_xray/val/',
    target_size=input_shape[:2],
    batch_size=16,
    class_mode='binary'
)


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [10]:
history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator,
        callbacks=callbacks
    )

Epoch 1/30
  1/326 [..............................] - ETA: 0s - loss: 0.7189 - accuracy: 0.3125WARNING:tensorflow:From /home/artur/PycharmProjects/conv-net-sandbox/venv/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
326/326 [==============================] - ETA: 0s - loss: 0.2129 - accuracy: 0.9103
Epoch 00001: val_loss improved from inf to 0.38139, saving model to /home/artur/PycharmProjects/conv-net-sandbox/logs/11-14-2020_09:37:40/ep001-loss0.2129-val_loss0.3814.h5
326/326 [==============================] - 157s 481ms/step - loss: 0.2129 - accuracy: 0.9103 - val_loss: 0.3814 - val_accuracy: 0.7500
Epoch 2/30
326/326 [==============================] - ETA: 0s - loss: 0.1108 - accuracy: 0.9613
Epoch 00002: val_loss improved from 0.38139 to 0.34817, saving model to /home/artur/PycharmProjects

In [ ]:
model.save(MODEL_OUTPUT_PATH)